In [2]:
# NumPy
import numpy as np

# Dataframe operations
import pandas as pd

# XGBoost
import xgboost as xgb

# Data visualization
import seaborn as sns
import matplotlib.pyplot as plt

# Scalers
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

#dataset
from sklearn.datasets import load_boston

# Models
from sklearn.linear_model import LogisticRegression #logistic regression
from sklearn.linear_model import Perceptron
from sklearn import svm #support vector Machine
from sklearn.ensemble import RandomForestClassifier #Random Forest
from sklearn.neighbors import KNeighborsClassifier #KNN
from sklearn.naive_bayes import GaussianNB #Naive bayes
from sklearn.tree import DecisionTreeClassifier #Decision Tree
from sklearn.model_selection import train_test_split #training and testing data split
from sklearn import metrics #accuracy measure
from sklearn.metrics import confusion_matrix #for confusion matrix
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier

# Cross-validation
from sklearn.model_selection import KFold #for K-fold cross validation
from sklearn.model_selection import cross_val_score #score evaluation
from sklearn.model_selection import cross_val_predict #prediction
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split

# GridSearchCV
from sklearn.model_selection import GridSearchCV

#Common Model Algorithms
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process

#Common Model Helpers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics

#Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.plotting import scatter_matrix
import warnings
import math
warnings.filterwarnings('ignore')

In [4]:
# CSVを読み込む
train= pd.read_csv("train.csv")
test= pd.read_csv("test.csv")

# データの統合
data = pd.concat([train, test], ignore_index = True)

# 提出用に
row_id = test["row_id"]

data["time"] = data['time'].str[-8:-3]
data

,row_id,time,x,y,direction,congestion
0,0,00:00,0,0,EB,70.0
1,1,00:00,0,0,NB,49.0
2,2,00:00,0,0,SB,24.0
3,3,00:00,0,1,EB,18.0
4,4,00:00,0,1,NB,60.0
...,...,...,...,...,...,...
851170,851170,23:40,2,3,NB,NaN
851171,851171,23:40,2,3,NE,NaN
851172,851172,23:40,2,3,SB,NaN
851173,851173,23:40,2,3,SW,NaN


In [5]:

# 全体の欠損データの個数確認
data_null = data.fillna(np.nan)
data_null.isnull().sum()

row_id           0
time             0
x                0
y                0
direction        0
congestion    2340
dtype: int64

In [6]:
data.describe()

,row_id,x,y,congestion
count,851175.000000,851175.000000,851175.000000,848835.000000
mean,425587.000000,1.138462,1.630769,47.815305
std,245713.202025,0.801478,1.089379,16.799392
min,0.000000,0.000000,0.000000,0.000000
25%,212793.500000,0.000000,1.000000,35.000000
50%,425587.000000,1.000000,2.000000,47.000000
75%,638380.500000,2.000000,3.000000,60.000000
max,851174.000000,2.000000,3.000000,100.000000


In [27]:
data1 = data[['time','x','y','direction','congestion']]
data1_dummies=pd.get_dummies(data1)
data1_dummies.head(3)

,x,y,congestion,time_00:00,time_00:20,time_00:40,time_01:00,time_01:20,time_01:40,time_02:00,...,time_23:20,time_23:40,direction_EB,direction_NB,direction_NE,direction_NW,direction_SB,direction_SE,direction_SW,direction_WB
0,0,0,70.0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,49.0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0,0,24.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [28]:
model_train = data1_dummies[:848835]
model_test = data1_dummies[848835:]

X = model_train.drop('congestion', axis=1)
Y = pd.DataFrame(model_train['congestion'])
x_test = model_test.drop('congestion', axis=1)

In [52]:
params = {'objective':'binary:logistic',
          'max_depth':5,
          'eta': 0.1, 
          'min_child_weight':1.0,
          'gamma':0.0,
          'colsample_bytree':0.8}

num_round = 1000

logloss = []
accuracy = []

kf = KFold(n_splits=4, shuffle=True, random_state=0)
for train_index, valid_index in kf.split(X):
    x_train, x_valid = X.iloc[train_index], X.iloc[valid_index] 
    y_train, y_valid = Y.iloc[train_index], Y.iloc[valid_index]
    
    dtrain = xgb.DMatrix(x_train, label=y_train)
    dtest = xgb.DMatrix(x_valid, label=y_valid)
    
    xgb_params = {
        # 回帰問題
        'objective': 'reg:linear',
        # 学習用の指標 (RMSE)
        'eval_metric': 'rmse',
    }
    evals = [(dtrain, 'train'), (dtest, 'eval')]
    evals_result = {}
    bst = xgb.train(xgb_params,
                    dtrain,
                    num_boost_round=50000,
                    early_stopping_rounds=15,
                    evals=evals,
                    evals_result=evals_result,
                    )

    y_pred = bst.predict(dtest)
    mse = mean_squared_error(y_valid, y_pred)
    print('RMSE:', math.sqrt(mse))
    
    

MemoryError: Unable to allocate 398. MiB for an array with shape (82, 636626) and data type int64

In [68]:
#predictで予測
dtest = xgb.DMatrix(x_test)
y_pred = bst.predict(dtest)
submission = pd.DataFrame({'row_id':test['row_id'], 'congestion':y_pred})
submission.to_csv('submission1.csv', index=False)